In [1]:
import json

# import the smartpynector module
from NBs import smartpynector as sp

In [2]:
# Defining constants
# Knowledge engine REST API URL
KNOWLEDGE_ENGINE_URL = "http://127.0.0.1:8280/rest"
SUBSCRIBER_ID = "http://127.0.0.1:8000/subscriber"

In [3]:
# Defining the smart connector
smart_connector = sp.SmartConnector(
    knowledgeBaseId=SUBSCRIBER_ID,
    knowledgeBaseName="Subscriber2",
    knowledgeBaseDescription="Subscriber2",
    reasonerEnabled=False,
)

In [4]:
sp.create_smart_connector(smart_connector, KNOWLEDGE_ENGINE_URL)

2023-05-03 17:19:10 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-03 17:19:13 DEBUG http://127.0.0.1:8280 "POST /rest/sc HTTP/1.1" 200 0
2023-05-03 17:19:13 INFO Request successful
2023-05-03 17:19:13 DEBUG 


<Response [200]>

In [5]:
GRAPH_PATTERN= '''?meas rdf:type saref:Measurement .
         ?meas saref:hasValue ?temp .
         ?meas saref:isMeasuredIn saref:TemperatureUnit .
         ?meas saref:hasTimestamp ?timestamp .
         ?meas saref:isMeasurementOf ?room_id .
         ?meas saref:relatesToProperty saref:Temperature .
         ?meas saref:measurementMadeBy ?device_id .'''

In [6]:
ask_ki = sp.register_knowledge_interaction(
    knowledge_interaction_type="AskKnowledgeInteraction",
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    prefixes={
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "saref": "https://w3id.org/saref#",
    },
    graph_pattern=GRAPH_PATTERN,
    headers={
        "Content-Type": "application/json",
        "knowledge-Base-Id": SUBSCRIBER_ID,
    },
)
ask_ki

2023-05-03 17:19:13 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-03 17:19:14 DEBUG http://127.0.0.1:8280 "POST /rest/sc/ki HTTP/1.1" 200 110
2023-05-03 17:19:14 INFO Request successful
2023-05-03 17:19:14 DEBUG {"knowledgeInteractionId":"http://127.0.0.1:8000/subscriber/interaction/8fc803d9-48e3-42af-bb3d-d75a8d2caf1c"}


{'knowledgeInteractionId': 'http://127.0.0.1:8000/subscriber/interaction/8fc803d9-48e3-42af-bb3d-d75a8d2caf1c',
 'prefixes': {'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
  'saref': 'https://w3id.org/saref#'}}

In [7]:
binding_set = sp.perform_ask_knowledge_interaction(
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    knowledge_base_id=SUBSCRIBER_ID,
    knowledge_interaction_id=ask_ki["knowledgeInteractionId"],
    ask_data=[]
).json()["bindingSet"]

2023-05-03 17:19:14 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-03 17:19:14 DEBUG http://127.0.0.1:8280 "POST /rest/sc/ask HTTP/1.1" 200 825
2023-05-03 17:19:14 INFO Request successful
2023-05-03 17:19:14 DEBUG {"bindingSet":[{"room_id":"\"http://127.0.0.1:8000/thermostat/rooms/1\"","temp":"20","device_id":"\"http://127.0.0.1:8000/thermostat/devices/1\"","meas":"<http://127.0.0.1:8000/thermostat/measurements/1>","timestamp":"\"2023-05-03T17:19:14+02:00\""}],"exchangeInfo":[{"initiator":"knowledgeBase","knowledgeBaseId":"http://127.0.0.1:8000/thermostat","knowledgeInteractionId":"http://127.0.0.1:8000/thermostat/interaction/ebde60ac-961e-484f-bb25-545d528d25e1","exchangeStart":"2023-05-03T15:19:14.707+00:00","exchangeEnd":"2023-05-03T15:19:14.944+00:00","status":"SUCCEEDED","bindingSet":[{"room_id":"\"http://127.0.0.1:8000/thermostat/rooms/1\"","temp":"20","device_id":"\"http://127.0.0.1:8000/thermostat/devices/1\"","meas":"<http://127.0.0.1:8000/thermostat/measuremen

In [8]:
READ_URL = "https://graphdb.odissei.nl/repositories/MateuszTest"
WRITE_URL = "https://graphdb.odissei.nl/repositories/MateuszTest/statements"

sp.store_data_in_graphdb(
    graph_pattern=GRAPH_PATTERN,
    binding_set=binding_set,
    prefixes=ask_ki["prefixes"],
    read_url=READ_URL,
    write_url=WRITE_URL,
)

HTTPError: HTTP Error 500: Internal Server Error